In [217]:
def read_packet_data(filename):
    
    with open(filename) as infile:
        raw_data = [line.strip() for line in infile]
    
    raw_data.append("")
    
    packet_pairs = list()
    for i in range(0, len(raw_data)):
        if len(raw_data[i])==0:
            packet_pairs.append([eval(raw_data[i-2]), eval(raw_data[i-1])])
            
    return packet_pairs


In [232]:
def compare_packets(packet_pair, undecided = True, right_order = None, verbose=False):
    
    left = packet_pair[0]
    right = packet_pair[1]

    
    if verbose:
        print(f" - Compare {left} vs {right}")
        
    if type(left) == int and type(right) == int:
        if left < right:
            right_order = True
            undecided = False   
            if verbose:
                print(f" - Left side is smaller, so inputs are in the right order")            
        elif left > right:
            right_order = False
            undecided = False
            if verbose:
                print(f" - Right side is smaller, so inputs are not in the right order")
    else:
        while undecided and left:
            if (type(left) == int and type(right) == list) or (type(left) == list and type(right) == int):
                if type(left) == int:
                    left = [left]
                    if verbose:
                        print(f" - Mixed types; convert left to {left} and retry comparison")
                else:
                    right = [right]
                    if verbose:
                        print(f" - Mixed types; convert right to {right} and retry comparison")
                    
            if len(left) > 0 and len(right) == 0:
                undecided = False
                right_order = False
                if verbose:
                    print("- Right side ran out of items, so inputs are not in the right order")    
                break
             
            undecided, right_order = compare_packets([left.pop(0), right.pop(0)], undecided, right_order, verbose)
            if not undecided:
                break   
        
        if undecided and (type(left) == list and len(left) == 0) and (type(right)==int or len(right) > 0):
            undecided = False
            right_order = True
            if verbose:
                print("- Left side ran out of items, so inputs are in the right order")
            
    return undecided, right_order

def compare_all(packet_pair_inputs):
    sum_indices = 0
    i = 1
    for packets_in in packet_pair_inputs:
        _, right_order = compare_packets(packets_in)
        if right_order:
            sum_indices += i
            #print(sum_indices)
        
        i += 1
        
    return sum_indices


packet_pairs = read_packet_data("inputs/day13-example.txt");
#_, right_order = compare_packets(packet_pairs[-19], verbose=True);
#print(f"right_order: {right_order}")
compare_all(packet_pairs)
# wrong: 5641 (too low)

1
2
4
6


13

In [234]:
test_pair = [[[1],1], [1]]
_, right_order = compare_packets(test_pair, verbose=True);


 - Compare [[1], 1] vs [1]
 - Compare [1] vs 1
 - Mixed types; convert right to [1] and retry comparison
 - Compare 1 vs 1
- Right side ran out of items, so inputs are not in the right order
